<a href="https://colab.research.google.com/github/aritraghsh09/GaMPEN/blob/master/tutorials/Making_predictions_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Predictions with GaMPEN

In this Jupyter Notebook, we will demonstrate how trained GaMPEN models can be used to make predictions on galaxy images. For an extensive documentation on GaMPEN, please refer to https://gampen.readthedocs.io/en/latest/index.html

This tutorial has been developed by [Aritra Ghosh](http://ghosharitra.com) and [Aayush Mishra](https://github.com/aayush2505).

## Preliminary stuff

### Connect to GPU runtime

Before we dive in, note that Google Colab gives you the ability to use GPUs for this tutorial.  

To do this, on the Google Colab menu bar select "Runtime" -> "Change runtime type" -> "Hardware accelerator" -> "GPU".

### Installing GaMPEN

First, let's install GaMPEN. 

Some of these commands are specifically for Google Colab. If doing this on your own machine, please follow the steps outlined [here](https://gampen.readthedocs.io/en/latest/Getting_Started.html)

In [ ]:
!git clone https://github.com/aritraghsh09/GaMPEN.git

In [ ]:
%cd /content/GaMPEN/

!pip install -r requirements.txt 

### Checking the Installation

In order to check whether the installation has occured succesfully, run the command below.  

In [ ]:
!make check

As long as the tests do not produce any errors, you are good to go! 

Note that warnings and and tests being skipped are ok! 

## Getting Images & Models to Perform Prediction

### Downloading Images and Creating the Necessary Folder Structure

Let's get some Hyper Suprime Cam images from its 2nd Public Data Release that we can use to perform predictions. For this demo, we will use the images from Fig. 13 of [Ghosh et. al. 2022](https://arxiv.org/pdf/2212.00051.pdf)

**⚠ NOTE:** In order for GaMPEN to work, you need to point it to a `/data/` folder which has all the key components it needs (this folder doesn't necessarily need to be name).  In the data folder, at the very least, you should have 
  * a `/cutouts/` folder with all the images on which you want to perform analysis
  * an `info.csv` file with file names of the various images.

In [ ]:
# Let's create two data-directories in our Colab runtime
# One containing low-z images and one containing mid-z images. 
%cd /content/
!mkdir data_lowz
!mkdir data_lowz/cutouts/
!mkdir data_midz
!mkdir data_midz/cutouts/

In [ ]:
#Now let's get some contents for these folders from the Yale FTP Servers

%cd /content/data_lowz/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/gampen_tutorial_files/for_preds/data_lowz/info.csv 

%cd /content/data_lowz/cutouts/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/gampen_tutorial_files/for_preds/data_lowz/cutouts/*.fits

%cd /content/data_midz/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/gampen_tutorial_files/for_preds/data_midz/info.csv 

%cd /content/data_midz/cutouts/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/gampen_tutorial_files/for_preds/data_midz/cutouts/*.fits

### Downloading the Trained Models

Now, let's download the models we trained on Hyper Suprime Cam Imaging in [Ghosh et. al. 2022](https://arxiv.org/pdf/2212.00051.pdf). Since our images are from the low and mid redshift bins, we download the corresponding trained models. 

In [ ]:
%cd /content/
%mkdir trained_models
%cd trained_models

!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/g_0_025/trained_model/g_0_025_real_data.pt
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/r_025_050/trained_model/r_025_050_real_data.pt

**⚠ NOTE:** Details about all the publicly-available GaMPEN models as well as the morphological catalogs for all galaxies in [Ghosh et. al. 2022](https://arxiv.org/pdf/2212.00051.pdf) can be found in our [Public Data Release Handbook](https://gampen.readthedocs.io/en/latest/Public_data.html)

### Inspecting the Files


Now, let's take a quick look at the downloaded info.csv files.

In [ ]:
import pandas as pd

info_lowz = pd.read_csv("/content/data_lowz/info.csv")
info_midz = pd.read_csv("/content/data_midz/info.csv")

In [ ]:
info_lowz

In [ ]:
info_midz

As outlined before, the only mandatory coulmn in the info.csv file is a column called `file_name` with the full name of each image file in the `/cutouts/` folder.


Now, let's take a quick look at the images. Note that HSC's cutout tool sometimes returns images with slightly different cutout-sizes (in terms of pixels). Thus, if you are using our models for performing inference, we recommend downloading cutouts that are at least 250x250 pixels. Then, using the correct options in GaMPEN, these will be cropped automatically while performing inference. 

Our pre-trained models accept the following sizes as input sizes for their images:-

* Low-z: $239 \times 239$ pixels
* Mid-z: $143 \times 143$ pixels
* High-z: $96 \times 96$ pixels

Inspite of the fact the GaMPEN automatically crops galaxies, we still start out with different sizes for each redshift bin as training & inference time goes down drastically with reduction in image size. Thus, it makes no senese to start with the same sizes for the low- and high-z bins. Each size above is chosen such that most galaxies at these redshifts will have $10 \times R_e < $ cutout size. 

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
import pylab as plt
import matplotlib as mpl
from astropy.io import fits

LOGMIN = 1e-3

#Defining a function to crop images
def crop_center(img, cropx, cropy):
    
    #Function from 
    #https://stackoverflow.com/questions/39382412/crop-center-portion-of-a-numpy-image
    
    y, x, *_ = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)    
    return img[starty:starty + cropy, startx:startx + cropx, ...]

In [ ]:
#Low-z Data
fig, ax = plt.subplots(1,2,figsize=(7,3))

for i,img_num in enumerate([45849776612206843, 71112151477012948]):
  img_data = fits.getdata('/content/data_lowz/cutouts/' + str(img_num) + '.fits')

  #Now, let's crop the image to it's starting size
  img_data = crop_center(img_data, 239, 239)

  ax[i].imshow(img_data,norm=mpl.colors.LogNorm(vmin=max(img_data.min(),LOGMIN)))

In [ ]:
#Mid-z Data
fig, ax = plt.subplots(1,2,figsize=(7,3))

for i,img_num in enumerate([38544642832100143, 40163802552993641]):
  img_data = fits.getdata('/content/data_midz/cutouts/' + str(img_num) + '.fits')

  #Now, let's crop the image to it's starting size
  img_data = crop_center(img_data, 143, 143)

  ax[i].imshow(img_data,norm=mpl.colors.LogNorm(vmin=max(img_data.min(),LOGMIN)))

## Running the Inference module

Now using the downloaded models and images, let's perform inference.

### Low-z

First, let's use the lowz model to perform inference on the lowz images.

For performing inference, first we need to create a directory to store the output files. Let's call this directory `bayesian_inference_runs`.

Additionally since we will be using models that have already been trained, we will be making predictions for the following three variables:-

$$ \left( \log\frac{L_B/L_T}{1-L_B/L_T}, \log R_e, \log \mathrm{Flux} \right) $$

The variables were additionally standard-scaled (i.e., mean subtracted and variance set to 1) during training (for more details see Section 4.1 of [Ghosh et. al. 2022](https://arxiv.org/pdf/2212.00051.pdf)). Now, in order to perform the inverse scaling properly, we need access to the different training values of these variables that were used to perform the forward scaling.

We make a separate directory called `scaling_data_lowz` and `scaling_data_midz` to place the relevant `info.csv` and `/splits/` files from the Yale FTP Servers. These were the files used for training and will help us unscale the predicted values back to ($L_B/L_T$,$R_e$, Flux). You should use the same options for these options if you are using our pre-trained models (as well as the corresponding options for the mid and high-z models)

In [ ]:
%cd /content/
%mkdir bayesian_inference_runs_lowz
%mkdir scaling_data_dir_lowz
%mkdir scaling_data_dir_lowz/splits/

%cd /content/scaling_data_dir_lowz/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/g_0_025/scaling_data_dir/info.csv

%cd /content/scaling_data_dir_lowz/splits/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/g_0_025/scaling_data_dir/splits/*.csv

The backbone of performing inference is the `inference.py` file at `/GaMPEN/ggt/modules/`.

In order to use this file, we run it by passing different variables to the inferece file. In order to understand the various options that can be specified while running inference you can type `!python GaMPEN/ggt/modules/inference.py --help` in a Google Colab code cell or consult the documentation [here](https://gampen.readthedocs.io/en/latest/Using_GaMPEN.html#inference). 

****⚠ STOP: We strongly recommend that you go through the page linked above to understand the various options we have used for performing inference below.**. 

Since the data we are using for inference doesn't have any ground-truth data available, we set `split` and `slug` to `None` and pass the `--no-labels` option to specify that we don't have any labels available for these images. 

In order to unscale the predictions back, we specify the path to the appropriate `scaling_data_dir` that we created in the previous step. We specify the `scaling_slug` as `balanced-dev2` since this was the option that was used for the pre-trained model and the scaling file we are using is named `balanced-dev2-train.csv`. All that is done using this file is that the full range of values for the prediction columns specified are read in --> their mean and standard deviation is determined --> and this is then used for unscaling the predicted values. If you are using your own trained models, you should of course point this to the relevant file in `/splits/` in your trainin data directory. 

The `--mc-dropout` and `--cov-errors` options specify that we want to perform both Monte Carlo dropout during inference as well include aleatoric errors in each of the Monte Carlo runs. The `n_runs` parameter controls the number of different Monte Carlo models generated for prediction. For a robust analysis, we recommend setting this to `500` or `1000`. We set this to `50` here just for demonstrative purposes. 



In [ ]:
%cd /content/

!python GaMPEN/ggt/modules/inference.py\
  --model_path='trained_models/g_0_025_real_data.pt'\
  --output_path="bayesian_inference_runs_lowz/"\
  --data_dir='data_lowz/'\
  --split=None\
  --slug=None\
  --cutout_size=239\
  --normalize\
  --transform\
  --n_workers=2\
  --parallel\
  --label_cols='custom_logit_bt,ln_R_e_asec,ln_total_flux_adus'\
  --model_type='vgg16_w_stn_oc_drp'\
  --repeat_dims\
  --channels=3\
  --label_scaling='std'\
  --mc_dropout\
  --cov_errors\
  --dropout_rate=0.0004\
  --n_runs=50\
  --no-labels\
  --scaling_data_dir='scaling_data_dir_lowz/'\
  --scaling_slug='balanced-dev2'


Note that output `.csv` files are stored in the output directory we specified. As many `.csv` files are created as the number of runs specified in `n_runs` and are named `inf_xx.csv`. Each file corresponds to the values predicted using a different model created using Monte Carlo Dropout. Each file will contain one sampled value from the predicted distribution by that model.

Let's take a look at `inf_1.csv`

In [ ]:
import pandas as pd

inf_1 = pd.read_csv('/content/bayesian_inference_runs_lowz/inf_1.csv')
inf_1

As expected we have the predicted values for each columns specified during inference. Note that these have already been un-scaled in terms of the the minmax/standard scaling specified. However, they are still in the format 

$$ \left( \log\frac{L_B/L_T}{1-L_B/L_T}, \log R_e, \log \mathrm{Flux} \right) $$


Now, we will use the [`result_aggregator.py`](https://github.com/aritraghsh09/GaMPEN/blob/master/ggt/modules/result_aggregator.py) file in GaMPEN to collate all these .csvs. The `result_aggregator` module of GaMPEN will collect all the csvs; scale the values back to $L_B/L_T$, $R_e$, Flux; produce summary statistics; as well as produce PDFs of the output variables for each image. 

**For an understanding of all the options available in the `result_aggregator` module, please refere to [this page.](https://gampen.readthedocs.io/en/latest/Using_GaMPEN.html#result-aggregator)**

The `data_dir` must point to the directory with the output csvs and the `out_summary_df_path` to the location you want the summary data-frame to be stored. 

The `unscale` option here specifies that we want to perform the inverse logit and logarithmic scalings. The `scaling_data_dir` should again point to the scaling files used during training (see previous discussion).

In [ ]:
%cd /content/
%mkdir lowz_pdfs

!python /content/GaMPEN/ggt/modules/result_aggregator.py\
    --data_dir="bayesian_inference_runs_lowz/"\
    --num=50\
    --out_summary_df_path="bayesian_inference_runs_lowz/summary.csv"\
    --out_pdfs_path="lowz_pdfs/"\
    --unscale\
    --scaling_df_path="scaling_data_dir_lowz/info.csv"\
    --drop_old

Now, let's inspect the `summary.csv` file as well as the predicted PDFs. 


In [ ]:
import pandas as pd

summary_lowz = pd.read_csv('/content/bayesian_inference_runs_lowz/summary.csv')
summary_lowz

In [ ]:
print(summary_lowz.columns)

As can be seen in the summary file, for every prediction column we have the 

  * mean (_mean)
  * median (_median)
  * mode (_mode)
  * standard deviation (_std)
  * skewness (_skew)
  * kurtosis (_kurt)
  * $1\sigma$ confidence interval (_sig_ci)
  * $2\sigma$ confidence interval (_twosig_ci)
  * $3\sigma$ confidence interval (_threesig_ci)

for the predicted distribution. 

**⚠ STOP: Note that the `result_aggregator` module also converts flux to magnitudes; however this coversion assumes a photometric zeropoint that is only true for HSC. If you are using the `result_aggregator` module for some other survey, you should change this.**


### Mid-z

Now, let's repeat the same as above, but now using midz model on the midz images.

In [ ]:
%cd /content/
%mkdir bayesian_inference_runs_midz
%mkdir scaling_data_dir_midz
%mkdir scaling_data_dir_midz/splits/

%cd /content/scaling_data_dir_midz/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/r_025_050/scaling_data_dir/info.csv

%cd /content/scaling_data_dir_midz/splits/
!wget ftp://ftp.astro.yale.edu/pub/hsc_morph/r_025_050/scaling_data_dir/splits/*.csv

Note that below we have updated all variables according the mid-z model. 

In [ ]:
%cd /content/

!python GaMPEN/ggt/modules/inference.py\
  --model_path='trained_models/r_025_050_real_data.pt'\
  --output_path="bayesian_inference_runs_midz/"\
  --data_dir='data_midz/'\
  --split=None\
  --slug=None\
  --cutout_size=143\
  --normalize\
  --transform\
  --n_workers=2\
  --parallel\
  --label_cols='custom_logit_bt,ln_R_e_asec,ln_total_flux_adus'\
  --model_type='vgg16_w_stn_oc_drp'\
  --repeat_dims\
  --channels=3\
  --label_scaling='std'\
  --mc_dropout\
  --cov_errors\
  --dropout_rate=0.0002\
  --n_runs=50\
  --no-labels\
  --scaling_data_dir='scaling_data_dir_midz/'\
  --scaling_slug='balanced-dev2'

/content
[2022-12-15 05:47:11,618] Performing pure inference without labels. Using
            column names to infer number of expected outputs.
            Split and Slug values entered will be ignored and
            info.csv will be used.
[2022-12-15 05:47:11,619] Loading images to device...
[2022-12-15 05:47:11,622] Generating PyTorch tensors from FITS files...
100% 2/2 [00:00<00:00, 9177.91it/s]
[2022-12-15 05:47:11,623] Preloading 2 tensors...
  0% 0/2 [07:28<?, ?it/s]

Aborted!


In [ ]:
%cd /content/
%mkdir midz_pdfs

!python /content/GaMPEN/ggt/modules/result_aggregator.py\
    --data_dir="bayesian_inference_runs_midz/"\
    --num=50\
    --out_summary_df_path="bayesian_inference_runs_midz/summary.csv"\
    --out_pdfs_path="midz_pdfs/"\
    --unscale\
    --scaling_df_path="scaling_data_dir_midz/info.csv"\
    --drop_old

In [ ]:
import pandas as pd

summary_midz = pd.read_csv('/content/bayesian_inference_runs_midz/summary.csv')
summary_midz

In [ ]:
summary_midz["preds_R_e_asec_sig_ci"][1].split(',')[0][1:]

In [ ]:
summary_midz["preds_R_e_asec_sig_ci"][1].split(',')[1][:-1]

## Plotting Results

Now, let's make a plot for the predicted distributions against the images.

### Importing required packages & Defining Functions

In [ ]:
!pip install matplotlib==3.1.3

import matplotlib as mpl
import numpy as np
import pylab as plt
import pandas as pd


import random
from matplotlib.patches import Rectangle
from matplotlib.ticker import FormatStrFormatter, ScalarFormatter
from astropy.io import fits 
LOGMIN=1e-4

In [ ]:
def plot_hists(summary_file_path,
               imgdir, pdf_dir,
               font_size=15,
               cutout_size=239):
  
  summary_df = pd.read_csv(summary_file_path)

  fig,ax1 = plt.subplots(len(summary_df),4,figsize=(4*5.3,len(summary_df)*4),
                          gridspec_kw={'width_ratios': [0.65, 1, 1, 1]})
  fig.subplots_adjust(wspace=0.12,hspace=0.23)

  row_counter = 0

  for i, img_num in enumerate(summary_df["object_id"]):
        
    ax = ax1[row_counter]
    ax[0].set_xticks([])
    ax[0].set_yticks([])
        
    img_data = fits.getdata(imgdir
                            + str(img_num) + ".fits")
    img_data = crop_center(img_data,cutout_size,cutout_size)
    ax[0].imshow(img_data,norm=mpl.colors.LogNorm(vmin=max(img_data.min(),LOGMIN)))

    pred_arr = np.load(pdf_dir + str(img_num) + ".npy")
  
    pred_cols = ["preds_bt","preds_R_e_asec","preds_total_mag"]
    pred_arr_idxs = [2,0,3] #indexes of columns in pred_arr

    for j, column_name in enumerate(pred_cols):

      ax[j+1].plot(pred_arr[pred_arr_idxs[j]],
                   pred_arr[pred_arr_idxs[j]+4],
                   label="PDF",lw=3)
      

      mode = summary_df[column_name + "_mode"][i]
      sig_ci = summary_df[column_name + "_sig_ci"][i]
      sig_ci = (float(sig_ci.split(',')[0][1:]),
                float(sig_ci.split(',')[1][:-1]))
      twosig_ci = summary_df[column_name + "_twosig_ci"][i]
      twosig_ci = (float(twosig_ci.split(',')[0][1:]),
                   float(twosig_ci.split(',')[1][:-1]))
      threesig_ci = summary_df[column_name + "_threesig_ci"][i]
      threesig_ci = (float(threesig_ci.split(',')[0][1:]),
                     float(threesig_ci.split(',')[1][:-1]))
      n_out = pred_arr[pred_arr_idxs[j]+4]
      
      ax[j+1].plot([mode,mode],[0,np.max(pred_arr[pred_arr_idxs[j]+4])],c='r',
                      ls='solid',label="Mode", lw =3) #plotting a x = Mode line
            
            
            
      rect = Rectangle((sig_ci[0], 0), sig_ci[1]-sig_ci[0], 0.25*np.max(n_out),color='coral',
                             alpha=0.5,label="68.27 %ile",lw=1)
      border = Rectangle((sig_ci[0], 0), sig_ci[1]-sig_ci[0], 0.25*np.max(n_out),ec='coral',
                             lw=3,fill=False)
      ax[j+1].add_patch(rect)  
      ax[j+1].add_patch(border)
    
      rect = Rectangle((twosig_ci[0], 0), twosig_ci[1]-twosig_ci[0], 0.175*np.max(n_out),color='goldenrod',
                             alpha=0.5,label="95.45 %ile")
      border = Rectangle((twosig_ci[0], 0), twosig_ci[1]-twosig_ci[0], 0.175*np.max(n_out),ec='goldenrod',
                             lw=3,fill=False)
      ax[j+1].add_patch(rect)
      ax[j+1].add_patch(border)
    
      rect = Rectangle((threesig_ci[0], 0), threesig_ci[1]-threesig_ci[0], 0.10*np.max(n_out),
                             color='seagreen',alpha=0.5,label="99.73 %ile")
      border = Rectangle((threesig_ci[0], 0), threesig_ci[1]-threesig_ci[0], 0.10*np.max(n_out),ec='seagreen',
                             lw=3,fill=False)
      ax[j+1].add_patch(rect)
      ax[j+1].add_patch(border)
            
            
            
      #ax[j+1].ticklabel_format(axis='both',style='sci',scilimits=(0,0))
      ax[j+1].set_yticks([])
      ax[j+1].tick_params(axis='x',labelsize=font_size-3)
            
                
    if row_counter == 0:
            ax[1].legend(loc='upper right',prop={'size': font_size-4})
            
                
              
    row_counter += 1


  ax[1].set_xlabel(r"$L_B/L_T$",fontsize=font_size)
  ax[2].set_xlabel(r"$R_e$ (asec)",fontsize=font_size)
  ax[3].set_xlabel(r"mag (g-band)",fontsize=font_size)



### Making Plots

In [ ]:
plot_hists(imgdir="/content/data_lowz/cutouts/",
           summary_file_path="/content/bayesian_inference_runs_lowz/summary.csv",
           pdf_dir="/content/lowz_pdfs/",
           font_size=18)

In [ ]:
plot_hists(imgdir="/content/data_midz/cutouts/",
           summary_file_path="/content/bayesian_inference_runs_midz/summary.csv",
           pdf_dir="/content/midz_pdfs/",
           font_size=18,
           cutout_size=143)